In [ ]:
%pip install yfinance

In [ ]:
%pip install numpy==1.22.0

In [ ]:
%pip install --upgrade tensorflow

In [56]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
### EXTRACT IMPORTANT DATA ###
data1 = yf.download("GS", start="2022-01-01", end="2023-01-01")
data2 = yf.download("COKE", start="2017-01-01", end="2018-01-01")
data3 = yf.download("AMZN", start="2023-01-01", end="2024-01-01")
data4 = yf.download("CAT", start="2018-01-01", end="2019-01-01")
data5 = yf.download("ED", start="2015-01-01", end="2016-01-01")
datatest = yf.download("INTC", start="2021-01-01", end="2024-01-01")

# Adjusted Close price gives the effective price when factoring in dividends and such.
# We only need to use close price as we are predicted over a short period of time.
closing_prices1 = data1['Close']
closing_prices2 = data2['Close']
closing_prices3 = data3['Close']
closing_prices4 = data4['Close']
closing_prices5 = data5['Close']
test_prices = datatest['Close']
window_size = 10

feature_vectors = []
output_vectors = []
test_features = []
test_outputs = []

# Use a sliding window to create feature vectors for each day
# GS
for i in range(window_size, len(closing_prices1)):
    feature_vector = closing_prices1[i - window_size:i].values
    feature_vectors.append(feature_vector)
    output_vectors.append(closing_prices1.iloc[i].values)
# COKE
for i in range(window_size, len(closing_prices2)):
    feature_vector = closing_prices2[i - window_size:i].values
    feature_vectors.append(feature_vector)
    output_vectors.append(closing_prices2.iloc[i].values)
# AMZN
for i in range(window_size, len(closing_prices3)):
    feature_vector = closing_prices3[i - window_size:i].values
    feature_vectors.append(feature_vector)
    output_vectors.append(closing_prices3.iloc[i].values)
# CAT
for i in range(window_size, len(closing_prices4)):
    feature_vector = closing_prices4[i - window_size:i].values
    feature_vectors.append(feature_vector)
    output_vectors.append(closing_prices4.iloc[i].values)
# ED
for i in range(window_size, len(closing_prices5)):
    feature_vector = closing_prices5[i - window_size:i].values
    feature_vectors.append(feature_vector)
    output_vectors.append(closing_prices5.iloc[i].values)

# TEST INTC
for i in range(window_size, len(test_prices)):
    test_vector = test_prices[i - window_size:i].values
    test_features.append(feature_vector)
    test_outputs.append(test_prices.iloc[i].values)

X = np.array(feature_vectors)
y = np.array(output_vectors).flatten()
X_t = np.array(test_features)
y_y = np.array(test_outputs)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [65]:
### NORMALIZE DATA ###
X_scaled = []
y_scaled = []

# Loop through each feature vector and corresponding y value
for i in range(len(X)):
    feature_vector = X[i]
    target_value = y[i]
    
    scale_value = feature_vector[-1]
    
    # Scale the feature vector and the corresponding target
    X_scaled.append((feature_vector / scale_value))
    y_scaled.append((target_value / scale_value))

# Convert lists back to arrays for training
X_scaled = np.array(X_scaled)
y_scaled = np.array(y_scaled)

# print(X_scaled)
# print(y_scaled)

In [66]:
def create_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [67]:
# Split X and y into even and odd indices
X_train = X_scaled[::2]  # Even indices for training
y_train = y_scaled[::2]

X_val = X_scaled[1::2]  # Odd indices for validation
y_val = y_scaled[1::2]

In [68]:
model = create_mlp_model(X_scaled.shape[1])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=32, verbose=1)

Epoch 1/500
20/20 [==============================] - 1s 11ms/step - loss: 0.7254 - val_loss: 0.3891
Epoch 2/500
20/20 [==============================] - 0s 4ms/step - loss: 0.1404 - val_loss: 0.0062
Epoch 3/500
20/20 [==============================] - 0s 5ms/step - loss: 0.0122 - val_loss: 0.0011
Epoch 4/500
20/20 [==============================] - 0s 6ms/step - loss: 0.0020 - val_loss: 5.3833e-04
Epoch 5/500
20/20 [==============================] - 0s 4ms/step - loss: 6.8400e-04 - val_loss: 5.3786e-04
Epoch 6/500
20/20 [==============================] - 0s 5ms/step - loss: 5.6847e-04 - val_loss: 5.2493e-04
Epoch 7/500
20/20 [==============================] - 0s 4ms/step - loss: 5.5518e-04 - val_loss: 5.2548e-04
Epoch 8/500
20/20 [==============================] - 0s 4ms/step - loss: 5.5785e-04 - val_loss: 5.2620e-04
Epoch 9/500
20/20 [==============================] - 0s 4ms/step - loss: 5.5701e-04 - val_loss: 5.2366e-04
Epoch 10/500
20/20 [==============================] - 0s 4ms/ste

In [70]:
def evaluate_model(model, X_train, Y_train, X_val, Y_val):
    Y_train_pred = model.predict(X_train)
    Y_val_pred = model.predict(X_val)

    print("Training MSE:", mean_squared_error(Y_train, Y_train_pred))
    print("Validation MSE:", mean_squared_error(Y_val, Y_val_pred))
    # print("Test MSE:", mean_squared_error(Y_test, Y_test_pred))

    print("Training R-squared:", r2_score(Y_train, Y_train_pred))
    print("Validation R-squared:", r2_score(Y_val, Y_val_pred))
    # print("Test R-squared:", r2_score(Y_test, Y_test_pred))

    print(Y_val[60:65])
    print(Y_val_pred[60:65])

evaluate_model(model, X_train, y_train, X_val, y_val)

20/20 [==============================] - 0s 3ms/step
Training MSE: 0.00041943241282787965
Validation MSE: 0.00045069986989066595
Training R-squared: -0.27717528125245616
Validation R-squared: -0.3408606194737376
[[0.9884253 ]
 [0.9928667 ]
 [0.99778295]
 [0.97049803]
 [1.02514722]]
[[0.9898219 ]
 [0.99125403]
 [0.9900372 ]
 [0.9891935 ]
 [0.99186957]]
